# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [123]:
# Доска почета тех, благодаря кому я сдал это дз
# 5 семинар
# https://www.kaggle.com/liweicai/resnet-18-data-augmentation-with-torchvision
# https://www.youtube.com/watch?v=ovZ_54IUSU4
# https://www.programcreek.com/python/example/105102/torchvision.datasets.ImageFolder
# https://www.kaggle.com/appetukhov/simple-cnn-baseline-with-pytroch-for-beginners/data
# https://www.kaggle.com/darbin/ka-task-8-2-tiny-image-net-cnn-ru/data
# Работа была сделана в Kaggle Notebook
# Респект Никите Захарову, который залил на кеггл данные по URL https://www.kaggle.com/amiralex/cs-hse-hw2-ogo/

In [124]:
import os
import sys
from pathlib import Path
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import euclidean
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
import torch 
from torch import nn
import math
from torch import optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import random
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, CenterCrop, RandomHorizontalFlip, RandomAffine, Compose, Normalize, RandomHorizontalFlip, RandomErasing, RandomPerspective, RandomRotation, Resize, ToPILImage, Pad
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import CosineAnnealingLR

In [127]:
import gc 
try:
    del val_data, train_data, model, train_dataloader, val_dataloader
except:
    pass
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
gc.collect()

149

In [128]:
#check GPU
print(torch.cuda.is_available())
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

True
cuda


In [129]:
seed = 456
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

### Подготовка данных

In [130]:
train_data = ImageFolder(
    root='../input/cs-hse-hw2-ogo/dataset/dataset/train', transform=Compose(
        [
            Resize((256, 256)),
            CenterCrop((224, 224)),
            RandomHorizontalFlip(p=0.5),
            RandomPerspective(),
            RandomAffine(degrees=30, translate=(0.2, 0.2), shear=0.2),
            ToTensor(), 
            Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), 
        ]))
val_data = ImageFolder(
    root='../input/cs-hse-hw2-ogo/dataset/dataset/val', transform=Compose(
        [
            Resize((256, 256)),
            CenterCrop((224, 224)),
            ToTensor(), 
            Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), 
        ]))

Даталоудеры

In [131]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)

In [132]:
# Just very simple sanity checks
assert isinstance(train_data[0], tuple)
assert len(train_data[0]) == 2
assert isinstance(train_data[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [142]:
def train_one_epoch(model, data_loader, optimizer, return_losses, criterion, scheduler, device):
    model.train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    total_predictions = np.array([])#.reshape((0, ))
    total_labels = np.array([])#.reshape((0, ))
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            # Move Batch to GPU
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            # Update weights
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            # Update descirption for tqdm
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    if return_losses:
        return metrics, all_losses
    else:
        return metrics

In [134]:
def train(model, epochs, train_dataloader, val_dataloader, optimizer, criterion, device):
    all_train_losses = []
    epoch_train_losses = []
    epoch_eval_losses = []
    for epoch in range(epochs):
        # Train step
        print(f"Train Epoch: {epoch}")
        train_metrics, one_epoch_train_losses = train_one_epoch(
            model=model,
            data_loader=train_dataloader,
            optimizer=optimizer,
            return_losses=True,
            criterion=criterion,
            device=device
        )
        # Save Train losses
        all_train_losses.extend(one_epoch_train_losses)
        epoch_train_losses.append(train_metrics["loss"])
        # Eval step
        print(f"Validation Epoch: {epoch}")
        with torch.no_grad():
            validation_metrics, predicted_labels, true_labels = validate(
                model=model,
                data_loader=val_dataloader,
                criterion=criterion
            )
        print('Metrics of an epoch: ', validation_metrics)
        # Save eval losses
        epoch_eval_losses.append(validation_metrics["loss"])

In [135]:

def validate(model, data_loader, criterion, device="cuda:0"):
    model = model.eval()
    total_loss = 0
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])
    with tqdm(total=len(data_loader), file=sys.stdout) as prbar:
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})
    return metrics, total_predictions, total_labels
        

### Обучение модели, запуски экспериментов

In [137]:
from torchvision.models import resnet18
from torchvision.models import mobilenet_v2
model = resnet18(pretrained=True)
model.fc = nn.Linear(512, 200)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [138]:
for param in model.parameters():
    param.requires_grad = True

In [139]:
# model = model
optimizer = optim.AdamW(model.parameters(), lr=0.0001, amsgrad=True)
criterion = nn.CrossEntropyLoss()
n_epochs = 10
scheduler = CosineAnnealingLR(optimizer, 1)

Простой тест на проверку правильности написанного кода

In [145]:
all_losses, predicted_labels, true_labels = validate(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_data)
accuracy = (predicted_labels == val_data.targets).mean()
print("tests passed")

Loss: 0.8395 Accuracy: 81.25: 100%|██████████| 157/157 [00:32<00:00,  4.85it/s]  
tests passed


Запустить обучение можно в ячейке ниже.

In [140]:
train(model, n_epochs, train_dataloader, val_dataloader, optimizer, criterion, device)

Train Epoch: 0
Loss: 1.707 Accuracy: 65.625: 100%|██████████| 1563/1563 [12:15<00:00,  2.13it/s]  
Validation Epoch: 1
Loss: 1.5053 Accuracy: 68.75: 100%|██████████| 157/157 [00:42<00:00,  3.71it/s]  
Metrics of an epoch:  {'loss': 1.5573838478440691, 'accuracy': 0.6173}
Train Epoch: 2
Loss: 1.1698 Accuracy: 75.0: 100%|██████████| 1563/1563 [12:06<00:00,  2.15it/s]   
Validation Epoch: 2
Loss: 1.4367 Accuracy: 68.75: 100%|██████████| 157/157 [00:41<00:00,  3.78it/s]  
Metrics of an epoch:  {'loss': 1.4203944198644844, 'accuracy': 0.6448}
Train Epoch: 3
Loss: 1.5037 Accuracy: 62.5: 100%|██████████| 1563/1563 [12:19<00:00,  2.11it/s]   
Validation Epoch: 3
Loss: 1.3763 Accuracy: 75.0: 100%|██████████| 157/157 [00:41<00:00,  3.76it/s]   
Metrics of an epoch:  {'loss': 1.3589025200552243, 'accuracy': 0.6562}
Train Epoch: 4
Loss: 1.7282 Accuracy: 56.25: 100%|██████████| 1563/1563 [12:20<00:00,  2.11it/s]  
Validation Epoch: 4
Loss: 1.0493 Accuracy: 81.25: 100%|██████████| 157/157 [00:42<00:

### Проверка полученной accuracy

После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, реализуйте и запустите функцию `evaluate`. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

In [144]:
all_losses, predicted_labels, true_labels = validate(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_data)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за это задание составит {} баллов".format(min(10, (10 * (accuracy - 0.5)/ 0.3))))

Loss: 0.8395 Accuracy: 81.25: 100%|██████████| 157/157 [00:32<00:00,  4.88it/s]  
Оценка за это задание составит 6.123333333333333 баллов


### Отчёт об экспериментах 

так. 
* я пробовал mobilenet и несколько resnet -18, 101
* аугментацию 
* лучший оптимизатор - АдамВ с амсградом. СГД, Адам - не пошли
* лосс брал только кроссэнтропию
* батчи - лучший размер 64. меньше или больше - смерть((